In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../data/PrideAndPredjudice/character-tagging/PrideAndPredjudice.entities', delimiter='\t')
df

,COREF,start_token,end_token,prop,cat,text
0,223,9,17,NOM,PER,a single man in possession of a good fortune
1,224,23,24,NOM,PER,a wife
2,225,34,36,NOM,PER,such a man
3,225,40,40,PRON,PER,his
4,226,43,44,NOM,FAC,a neighbourhood
...,...,...,...,...,...,...
23490,6,145039,145039,PROP,GPE,Derbyshire
23491,2986,145047,145047,PRON,PER,them
23492,2992,145049,145056,NOM,FAC,CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO . ...
23493,74,145058,145059,PROP,FAC,CHANCERY LANE


In [3]:
df_characters = df[df.cat == 'PER']
df_characters

,COREF,start_token,end_token,prop,cat,text
0,223,9,17,NOM,PER,a single man in possession of a good fortune
1,224,23,24,NOM,PER,a wife
2,225,34,36,NOM,PER,such a man
3,225,40,40,PRON,PER,his
5,227,56,58,NOM,PER,the surrounding families
...,...,...,...,...,...,...
23486,2986,145018,145018,PRON,PER,them
23487,2986,145021,145021,PRON,PER,they
23488,2991,145031,145039,NOM,PER,"the persons who , by bringing her into Derbyshire"
23489,82,145037,145037,PRON,PER,her


In [4]:
len(df_characters.groupby(df.COREF))

2301

In [5]:
characters = {}

for coref_id, coref_df in df_characters.groupby(df.COREF):
    try:
        proper_noun_df = coref_df[coref_df.prop == 'PROP']
        if not proper_noun_df.empty:
            name = proper_noun_df.text[proper_noun_df.text.str.len() == proper_noun_df.text.str.len().max()].unique()[0]
            characters[coref_id] = name
    except:
        continue

characters

{2: 'the Netherfield family',
 4: 'Hertfordshire',
 7: 'the Longbourn',
 9: 'the Meryton',
 10: 'Lucas Lodge',
 12: 'Pemberley',
 14: 'January',
 15: 'Hunsford',
 17: 'Kent',
 20: 'Rosings',
 21: 'the ----shire',
 35: 'Matlock',
 37: 'Dovedale',
 50: 'Gretna Green',
 75: 'CHAPTER L. Mr. Bennet',
 76: 'Mrs. Long',
 77: 'Mr. Morris',
 78: 'Michaelmas',
 79: 'you,--Mr . Bingley',
 80: 'Sir William Lucas',
 81: 'Lady Lucas',
 82: 'the eldest Miss Bennet',
 83: 'poor Jane , who would willingly have gone through the world without believing that so much wickedness existed in the whole race of mankind as was here collected in one individual',
 84: 'Miss Lydia Bennet',
 85: 'Mrs. Bennet',
 86: 'Lady Catherine de Bourgh',
 87: 'Mary King',
 88: 'Mr. Hurst',
 89: 'that disagreeable Mr. Darcy',
 90: 'Mrs. Hurst',
 91: 'poor Miss Bingley',
 92: 'Miss Lucas',
 93: 'Miss King',
 94: 'Maria Lucas',
 95: 'Good God',
 96: 'Dear Lizzy',
 99: 'the younger Miss Bennets',
 100: 'the younger Miss Lucases',
 

In [6]:
from collections import defaultdict
import string


merged_idxs = set()
merged_coref_ids = defaultdict(list)

female_titles = [
    'Mrs.',
    'Lady',
    'Miss',
    'Aunt',
    'Duchess',
]

male_titles = [
    'Sir',
    'Mr.',
    'Lord',
    'Uncle',
    'Colonel',
]

neutral_titles_and_starters = [
    'Dr.',
    'Dear',
    'Poor',
    'Old',
    'Dearsest',
]


# Merge characters based on titles
for i, (coref_id, base_name) in enumerate(characters.items()):
    
    if i in merged_idxs:
        continue
    
    try:
        base_split_name = base_name.split()

        base_titles = []
        base_gender = 'neutral'
        while base_split_name[0] in male_titles + female_titles + neutral_titles_and_starters:

            if base_split_name[0] in male_titles:
                base_gender = 'male'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            elif base_split_name[0] in female_titles:
                base_gender = 'female'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            else:
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            
        base_forename = base_split_name[0]
        base_surname = base_split_name[-1]
            

        for j, (other_coref_id, other_name) in enumerate(characters.items()):
            
            if j in merged_idxs:
                continue
            
            if j <= i:
                continue
            other_split_name = other_name.split()
            
            other_titles = []
            other_gender = 'neutral'
            while any([name in male_titles + female_titles + neutral_titles_and_starters for name in other_split_name]):

                if other_split_name[0] in male_titles:
                    other_gender = 'male'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                elif other_split_name[0] in female_titles:
                    other_gender = 'female'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                else:
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])

            other_forename = other_split_name[0]
            other_surname = other_split_name[-1]
            
            if base_gender == 'male' and other_gender == 'female' or base_gender == 'female' and other_gender == 'male':
                continue
            
            if other_forename != other_surname and base_forename != base_surname:
                if base_forename != other_forename or base_surname != other_surname:
                    continue
                else:
                    merged_idxs.add(j)
                    merged_coref_ids[coref_id].append(other_coref_id)
                    print(f'Merging1: {base_name} -> {other_name}')
                    continue
            
            # Otherwise gender is the same/neutral, if the forenames match we should merge
            
            if other_forename == base_forename:
                merged_idxs.add(j)
                merged_coref_ids[coref_id].append(other_coref_id)
                print(f'Merging2: {base_name} -> {other_name}')
    except Exception as e:
        pass

Merging2: Lucas Lodge -> Lady Lucas
Merging2: Lucas Lodge -> Miss Lucas
Merging2: Poor Wickham -> Mrs. Wickham
Merging2: Mrs. Reynolds -> Poor Reynolds
Merging2: Lydia Wickham -> Poor Lydia


In [7]:
df_merged = df_characters[df_characters['COREF'].isin(list(characters.keys()))]
df_merged.COREF.unique().size

177

In [8]:
for coref_id, merged_coref_list in merged_coref_ids.items():
    for merged_coref_id in merged_coref_list:
        df_merged.loc[df_merged["COREF"] == merged_coref_id, "COREF"] = coref_id
    
df_merged.COREF.unique().size

172

In [9]:
min_mentions = 10

df_filtered = df_merged.groupby('COREF').filter(lambda x: len(x) > min_mentions)
df_filtered.COREF.unique().size

47

In [10]:
df_filtered

,COREF,start_token,end_token,prop,cat,text
12,75,80,81,PROP,PER,Mr. Bennet
18,75,105,106,PROP,PER,Mr. Bennet
19,75,109,109,PRON,PER,he
21,76,124,125,PROP,PER,Mrs. Long
23,76,132,132,PRON,PER,she
...,...,...,...,...,...,...
23476,12,144970,144970,PRON,PER,its
23479,82,144989,144989,PRON,PER,her
23484,147,145009,145009,PROP,PER,Darcy
23485,82,145014,145014,PROP,PER,Elizabeth


In [11]:
df_filtered['character_name'] = df_filtered.COREF.map(lambda x: characters[x])

In [12]:
df_filtered.to_csv('../data/PrideAndPredjudice/processed_characters.csv')

In [13]:
import json


merged_coref_to_new_coref = {}

for coref_id, merged_list in merged_coref_ids.items():
    for merged_id in merged_coref_list:
        merged_coref_to_new_coref[merged_id] = coref_id

with open('../data/PrideAndPredjudice/coref_merge_map.json', 'w') as f:
    json.dump(merged_coref_to_new_coref, f)